In [1]:
import numpy as np
import nibabel as nib
import SimpleITK as sitk
from scipy.spatial.distance import directed_hausdorff

In [2]:
# Load the NIfTI files
image1 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_27/GT2.nii.gz')
image2 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_67.nii.gz')

In [3]:
# Load NIfTI images
nii_1 = nib.load('ai4mi_project/data/segthor_train/train/Patient_27/GT2.nii.gz')
nii_2 = nib.load('ai4mi_project/data/segthor_train/train/Patient_67.nii.gz')

In [4]:
def compute_hausdorff(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Convert to numpy arrays
    array1 = sitk.GetArrayFromImage(img1)
    array2 = sitk.GetArrayFromImage(img2)

    # Get all unique labels 
    labels1 = np.unique(array1)
    labels2 = np.unique(array2)
    
    # Dictionary to store Hausdorff distance for each label
    hausdorff_distances = {}

    common_labels = labels1

    # Loop over each label (segmentation)
    for label in common_labels:
        
        if label == 0:  # Skip background
            continue
        
        # Create binary masks for the current label in both images
        mask1 = (array1 == label).astype(int)
        
        if label == 1:
            mask2 = (array2 == labels2[1]).astype(int)
        if label == 2:
            mask2 = (array2 == labels2[2]).astype(int)
        if label == 3:
            mask2 = (array2 == labels2[3]).astype(int)
        if label == 4:
            mask2 = (array2 == labels2[4]).astype(int)
        
        # Get the coordinates of non-zero points in the binary masks
        coords1 = np.column_stack(np.where(mask1))
        coords2 = np.column_stack(np.where(mask2))

        # Compute the directed Hausdorff distances
        hausdorff_distance_1_to_2 = directed_hausdorff(coords1, coords2)[0]
        hausdorff_distance_2_to_1 = directed_hausdorff(coords2, coords1)[0]
    
        # Symmetric Hausdorff distance (maximum of the two directed distances)
        hausdorff_distance = max(hausdorff_distance_1_to_2, hausdorff_distance_2_to_1)
    
        # Store the result
        hausdorff_distances[label] = hausdorff_distance
    
        print(f"Label {label}: Hausdorff Distance = {hausdorff_distance:.4f}")

    return sum(hausdorff_distances.values())

sum_haus = compute_hausdorff(image1, image2)
print("\nSum Hausdorff Distances: ",sum_haus)

Label 1: Hausdorff Distance = 5.9161
Label 2: Hausdorff Distance = 227.4269
Label 3: Hausdorff Distance = 6.7082
Label 4: Hausdorff Distance = 6.4031

Sum Hausdorff Distances:  246.4543192893994


In [6]:
def compute_volumetric(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Get the data arrays
    mask1 = img1.get_fdata()
    mask2 = img2.get_fdata()
    
    # Get all unique labels from both masks (excluding 0 for the background)
    labels_1 = np.unique(mask1[mask1 > 0])
    labels_2 = np.unique(mask2[mask2 > 0])
    
    # Get the voxel sizes from the headers
    voxel_size1 = np.prod(nii_1.header.get_zooms())  # in mm^3
    voxel_size2 = np.prod(nii_2.header.get_zooms())  # in mm^3
    
    # Function to compute volume for a given label
    def compute_volume(mask, label, voxel_size):
        return np.sum(mask == label) * voxel_size
    
    # Iterate over each label (organ) and compute Volumetric Similarity
    vs_results = {}
    
    for label in labels_1:
        # Compute volumes for the current label in both scans
        volume1 = compute_volume(mask1, label, voxel_size1)
    
        if label == 1:
            volume2 = compute_volume(mask2, labels_2[0], voxel_size2)
        if label == 2:
            volume2 = compute_volume(mask2, labels_2[1], voxel_size2)
        if label == 3:
            volume2 = compute_volume(mask2, labels_2[2], voxel_size2)
        if label == 4:
            volume2 = compute_volume(mask2, labels_2[3], voxel_size2)
    
    
        # Compute VS only if both volumes are non-zero
        if volume1 + volume2 > 0:
            vs = 1 - abs(volume1 - volume2) / (volume1 + volume2)
        else:
            vs = 0  # If both volumes are 0, there's no overlap
    
        # Store the result
        vs_results[label] = vs

        print(f"\nLabel {label}")
        print(f"Volume img1: {volume1}")
        print(f"Volume img2: {volume2}\n")
    
    # Print Volumetric Similarity results for each organ
    for label, vs in vs_results.items():
        print(f"Label {label}: Volumetric Similarity = {vs:.4f}")

    return sum(vs_results.values())/4

mean_vol = compute_volumetric(nii_1, nii_2)
print("\nMean Volumetric Similarity: ", mean_vol)


Label 1.0
Volume img1: 43016.43371582031
Volume img2: 43601.98974609375


Label 2.0
Volume img1: 845123.291015625
Volume img2: 824096.6796875


Label 3.0
Volume img1: 30998.22998046875
Volume img2: 33638.00048828125


Label 4.0
Volume img1: 215873.71826171875
Volume img2: 218750.0

Label 1.0: Volumetric Similarity = 0.9932
Label 2.0: Volumetric Similarity = 0.9874
Label 3.0: Volumetric Similarity = 0.9592
Label 4.0: Volumetric Similarity = 0.9934

Mean Volumetric Similarity:  0.983296217576162


In [7]:
def compute_confusion_metrics(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    seg1 = img1.get_fdata()
    seg2 = img2.get_fdata()
    
    labels_1 = np.unique(seg1[seg1 > 0])
    labels_2 = np.unique(seg2[seg2 > 0])
    my_dict = dict(zip(labels_1, labels_2))
    
    # Ensure both segmentations have the same shape
    assert seg1.shape == seg2.shape, "The two segmentations must have the same shape."
    
    # Convert the 3D arrays into 1D arrays for easy comparison
    seg1_flat = seg1.flatten()
    seg2_flat = seg2.flatten()
    
    for label in labels_1:
    
        # Compute True Positives, True Negatives, False Positives, and False Negatives
        TP = np.sum((seg1_flat == label) & (seg2_flat == my_dict[label]))  # Both true
        TN = np.sum((seg1_flat != label) & (seg2_flat != my_dict[label]))  # Both false
        FP = np.sum((seg1_flat != label) & (seg2_flat == my_dict[label]))  # False positive: seg2 says 1 but seg1 says 0
        FN = np.sum((seg1_flat == label) & (seg2_flat != my_dict[label]))  # False negative: seg2 says 0 but seg1 says 1
        
        # Total number of voxels
        total_voxels = seg1_flat.size
        
        # Calculate percentages
        TP_percent = (TP / total_voxels) * 100
        TN_percent = (TN / total_voxels) * 100
        FP_percent = (FP / total_voxels) * 100
        FN_percent = (FN / total_voxels) * 100
        
        # Output the results in percentages
        print(f'\nResults for Label: {label}, Total voxels for this label: {np.sum(seg1_flat == label)}')
        print(f'True Positives (TP): {TP_percent:.4f}%, TP voxels: {TP}')
        print(f'True Negatives (TN): {TN_percent:.4f}%, TN voxels: {TN}')
        print(f'False Positives (FP): {FP_percent:.4f}%, FP voxels: {FP}')
        print(f'False Negatives (FN): {FN_percent:.4f}%, FN voxels: {FN}')

compute_confusion_metrics(nii_1, nii_2)


Results for Label: 1.0, Total voxels for this label: 22553
True Positives (TP): 0.0331%, TP voxels: 18467
True Negatives (TN): 99.9517%, TN voxels: 55809726
False Positives (FP): 0.0079%, FP voxels: 4393
False Negatives (FN): 0.0073%, FN voxels: 4086

Results for Label: 2.0, Total voxels for this label: 443088
True Positives (TP): 0.1077%, TP voxels: 60119
True Negatives (TN): 98.5403%, TN voxels: 55021639
False Positives (FP): 0.6661%, FP voxels: 371945
False Negatives (FN): 0.6859%, FN voxels: 382969

Results for Label: 3.0, Total voxels for this label: 16252
True Positives (TP): 0.0274%, TP voxels: 15292
True Negatives (TN): 99.9667%, TN voxels: 55818076
False Positives (FP): 0.0042%, FP voxels: 2344
False Negatives (FN): 0.0017%, FN voxels: 960

Results for Label: 4.0, Total voxels for this label: 113180
True Positives (TP): 0.1892%, TP voxels: 105634
True Negatives (TN): 99.7811%, TN voxels: 55714438
False Positives (FP): 0.0162%, FP voxels: 9054
False Negatives (FN): 0.0135%, FN

In [8]:
def compute_average(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Convert to numpy arrays for easier manipulation
    seg1_np = sitk.GetArrayFromImage(img1)
    seg2_np = sitk.GetArrayFromImage(img2)
    
    # Find all unique labels (excluding background)
    labels1 = np.unique(seg1_np)
    labels2 = np.unique(seg2_np) 
    
    average_surface_distances = []
    labels = labels1[1:]
    
    for label in labels:
    
        # Extract binary masks for the current label in both segmentations
        seg1_mask = (seg1_np == label).astype(np.uint8)
        seg2_mask = (seg2_np == label).astype(np.uint8)
    
        if label == 1:
            seg2_mask = (seg2_np == labels2[1]).astype(np.uint8)
        if label == 2:
            seg2_mask = (seg2_np == labels2[2]).astype(np.uint8)
        if label == 3:
            seg2_mask = (seg2_np == labels2[3]).astype(np.uint8)
        if label == 4:
            seg2_mask = (seg2_np == labels2[4]).astype(np.uint8)
    
        # Convert the masks back to SimpleITK images
        seg1_mask_img = sitk.GetImageFromArray(seg1_mask)
        seg2_mask_img = sitk.GetImageFromArray(seg2_mask)
    
        # Compute Average Surface Distance
        seg1_surface = sitk.LabelContour(seg1_mask_img)
        seg2_surface = sitk.LabelContour(seg2_mask_img)
    
        surface_distance_filter = sitk.HausdorffDistanceImageFilter()
        surface_distance_filter.Execute(seg1_surface, seg2_surface)
        avg_distance = surface_distance_filter.GetAverageHausdorffDistance()
        average_surface_distances.append(avg_distance)
    
    # Output results for each label
    for i, label in enumerate(labels):
        print(f"Label {label}:")
        print(f"  Average Surface Distance: {average_surface_distances[i]}")

compute_average(image1, image2)

Label 1:
  Average Surface Distance: 0.8713971031274028
Label 2:
  Average Surface Distance: 32.09853647128438
Label 3:
  Average Surface Distance: 0.5357333845381963
Label 4:
  Average Surface Distance: 0.7775224238345334
